## Fine tune DialoGPT for E commerce customer service conversations

In [70]:
import torch

torch.cuda.is_available()

print(torch.__version__)

1.13.1+cpu


Hugging Face Dataset library

https://huggingface.co/docs/datasets/v2.14.4/en/index

In [44]:
from datasets import load_dataset

# Split ="train" to make it Dataset object, by default it load as Dataset Dict object
encoded_dataset = load_dataset('json', split='train', data_files="Dataset/Encoded_Ecommerce_FAQ_Chatbot_dataset.json")

Found cached dataset json (C:/Users/User/.cache/huggingface/datasets/json/default-b1a5f79678292222/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


In [45]:
encoded_dataset

Dataset({
    features: ['questions', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 110
})

In [46]:
type(encoded_dataset)

datasets.arrow_dataset.Dataset

In [47]:
# train, validation test split

splited_dataset = encoded_dataset.train_test_split(test_size=0.15)

Loading cached split indices for dataset at C:\Users\User\.cache\huggingface\datasets\json\default-b1a5f79678292222\0.0.0\8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96\cache-afdb24240a5283a3.arrow and C:\Users\User\.cache\huggingface\datasets\json\default-b1a5f79678292222\0.0.0\8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96\cache-826fb60ff4597ffb.arrow


In [48]:
splited_dataset

DatasetDict({
    train: Dataset({
        features: ['questions', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 93
    })
    test: Dataset({
        features: ['questions', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 17
    })
})

In [49]:
# with open("Dataset/Encoded_Ecommerce_FAQ_Chatbot_dataset.json", "r", encoding='utf-8') as json_file:
#     encoded_dataset = json.load(json_file)

# # Convert python list -> numpy array -> pytorch tensor
# for key in ['input_ids', 'attention_mask', 'labels']:
#     encoded_dataset[key] = torch.tensor(np.array(encoded_dataset[key]))

In [50]:
# encoded_dataset['train']

Hugging Face Transformer Library documentation

https://huggingface.co/docs/transformers/v4.31.0/en/main_classes/trainer#transformers.Trainer

dialoGPT model

DialoGPT (from Microsoft Research) released with the paper DialoGPT: Large-Scale Generative Pre-training for Conversational Response Generation by Yizhe Zhang, Siqi Sun, Michel Galley, Yen-Chun Chen, Chris Brockett, Xiang Gao, Jianfeng Gao, Jingjing Liu, Bill Dolan.

source : https://arxiv.org/abs/1911.00536

Microsoft github : https://github.com/microsoft/DialoGPT#retraining-full-models

In [51]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorWithPadding

In [52]:
model_name = "microsoft/DialoGPT-large"
# model_name = "microsoft/DialoGPT-medium"
# model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [53]:
tokenizer.pad_token = tokenizer.eos_token

In [54]:
max_token_limit = tokenizer.max_len_single_sentence
print(f"Maximum token limit for single sentence: {max_token_limit}")

Maximum token limit for single sentence: 1024


Hugging Face evaluate library

https://huggingface.co/docs/evaluate/index

Bleu score- https://huggingface.co/spaces/evaluate-metric/bleu

evaluate bleu expects :
predictions (list of strs): Translations to score.
references (list of lists of strs): references for each translation.

In [55]:
import evaluate

metrics = evaluate.combine(
    ["bleu", "meteor"], force_prefix=True
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = [str(prediction) for prediction in predictions]
    references = [[str(reference) for reference in reference_list] for reference_list in labels]

    # bleu = evaluate.load("bleu")
    # results = bleu.compute(predictions=predictions, references=references)

    results = metrics.compute(predictions=predictions, references=references)

    return results

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [56]:
# training_args = TrainingArguments(
#     output_dir="./output",
#     overwrite_output_dir=True,
#     num_train_epochs=5,
#     per_device_train_batch_size=1,
#     save_steps=1000,
#     save_total_limit=2,
#     optim="adamw_torch"
# )

https://huggingface.co/docs/transformers/main_classes/trainer

In [57]:
training_args = TrainingArguments(
    output_dir="./logs", # Output fine-tuned model and logs
    overwrite_output_dir=True, # Overwrite the output directory if exists
    num_train_epochs=5, # Number of training epochs (110 num rows, allow more epoch for less dataset)
    per_device_train_batch_size=2, # Batch size for training (number of samples in each batch)
    save_steps=100, # Number of steps before saving a checkpoint
    save_total_limit=1, # Maximum number of checkpoints to keep
    evaluation_strategy="steps", # Evaluate on validation data at specified steps
    eval_steps=100, # Evaluate every 100 steps
    logging_steps=25, # Log training information every n steps
    learning_rate=1e-4, # start with Low learning rate, dialoGPT paper use 1e-5 to 5e-5.
    # warmup_steps=500, # Warm-up steps for learning rate (helps with stability)
    weight_decay=0.01, # Weight decay (L2 regularization) parameter (prevent overfitting)
    load_best_model_at_end=True, # Load the best model checkpoint at the end of training, only works when evaluation strategy is steps
    metric_for_best_model="eval_loss", # Metric to determine best model
    optim="adamw_torch" # Optimizer for training - Adamw
)

In [58]:
data_collator_ = DataCollatorWithPadding(tokenizer=tokenizer)

In [59]:
splited_dataset['train']

Dataset({
    features: ['questions', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 93
})

In [60]:
validation_set = splited_dataset['test']

validation_set

Dataset({
    features: ['questions', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 17
})

In [61]:
from transformers import EarlyStoppingCallback

In [62]:
early_stop = EarlyStoppingCallback(3, 0.01)

In [63]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=splited_dataset['train'],
    eval_dataset=splited_dataset['test'],
    # tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
    # data_collator = data_collator_,
    callbacks=[early_stop] 
)

In [64]:
torch.cuda.empty_cache()

In [65]:
for key in ['input_ids', 'attention_mask', 'labels']:
    print(f"{key} length : {len(splited_dataset['train'][key])}")

input_ids length : 93
attention_mask length : 93
labels length : 93


In [67]:
trainer.train()

  0%|          | 0/235 [00:00<?, ?it/s]

{'loss': 1.127, 'learning_rate': 8.851063829787234e-05, 'epoch': 0.53}
{'loss': 0.6301, 'learning_rate': 7.787234042553192e-05, 'epoch': 1.06}
{'loss': 0.3117, 'learning_rate': 6.72340425531915e-05, 'epoch': 1.6}
{'loss': 0.2651, 'learning_rate': 5.659574468085107e-05, 'epoch': 2.13}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.3466397821903229, 'eval_bleu_bleu': 0.0, 'eval_bleu_precisions': [0.0, 0.0, 0.0, 0.0], 'eval_bleu_brevity_penalty': 1.0, 'eval_bleu_length_ratio': 71.0, 'eval_bleu_translation_length': 1207, 'eval_bleu_reference_length': 17, 'eval_meteor_meteor': 0.0, 'eval_runtime': 71.0908, 'eval_samples_per_second': 0.239, 'eval_steps_per_second': 0.042, 'epoch': 2.13}
{'loss': 0.3129, 'learning_rate': 4.595744680851064e-05, 'epoch': 2.66}
{'loss': 0.2174, 'learning_rate': 3.531914893617021e-05, 'epoch': 3.19}
{'loss': 0.1236, 'learning_rate': 2.468085106382979e-05, 'epoch': 3.72}
{'loss': 0.1352, 'learning_rate': 1.4042553191489363e-05, 'epoch': 4.26}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.3484913110733032, 'eval_bleu_bleu': 0.0, 'eval_bleu_precisions': [0.0, 0.0, 0.0, 0.0], 'eval_bleu_brevity_penalty': 1.0, 'eval_bleu_length_ratio': 71.0, 'eval_bleu_translation_length': 1207, 'eval_bleu_reference_length': 17, 'eval_meteor_meteor': 0.0, 'eval_runtime': 72.0661, 'eval_samples_per_second': 0.236, 'eval_steps_per_second': 0.042, 'epoch': 4.26}
{'loss': 0.0983, 'learning_rate': 3.4042553191489363e-06, 'epoch': 4.79}
{'train_runtime': 5657.304, 'train_samples_per_second': 0.082, 'train_steps_per_second': 0.042, 'train_loss': 0.34364249985268774, 'epoch': 5.0}


TrainOutput(global_step=235, training_loss=0.34364249985268774, metrics={'train_runtime': 5657.304, 'train_samples_per_second': 0.082, 'train_steps_per_second': 0.042, 'train_loss': 0.34364249985268774, 'epoch': 5.0})

In [68]:
import pandas as pd

pd.DataFrame(trainer.state.log_history)

,loss,learning_rate,epoch,step,eval_loss,eval_bleu_bleu,eval_bleu_precisions,eval_bleu_brevity_penalty,eval_bleu_length_ratio,eval_bleu_translation_length,eval_bleu_reference_length,eval_meteor_meteor,eval_runtime,eval_samples_per_second,eval_steps_per_second,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
0,1.1270,0.000089,0.53,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.6301,0.000078,1.06,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.3117,0.000067,1.60,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.2651,0.000057,2.13,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,2.13,100,0.346640,0.0,"[0.0, 0.0, 0.0, 0.0]",1.0,71.0,1207.0,17.0,0.0,71.0908,0.239,0.042,NaN,NaN,NaN,NaN,NaN
5,0.3129,0.000046,2.66,125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.2174,0.000035,3.19,150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.1236,0.000025,3.72,175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.1352,0.000014,4.26,200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,4.26,200,0.348491,0.0,"[0.0, 0.0, 0.0, 0.0]",1.0,71.0,1207.0,17.0,0.0,72.0661,0.236,0.042,NaN,NaN,NaN,NaN,NaN


In [69]:
model.save_pretrained("fine_tuned_dialogpt_FAQ_Ecommerce_large_2")
# tokenizer.save_pretrained("tokenizer")